<a href="https://colab.research.google.com/github/taninao1122/machine_learning/blob/master/tf2_train_mobile_net-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基本設定と定義

In [1]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q tf-nightly

     |████████████████████████████████| 390.1MB 46kB/s 
     |████████████████████████████████| 10.2MB 139kB/s 
     |████████████████████████████████| 460kB 42.3MB/s 


In [3]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import array_to_img, img_to_array, load_img

import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras import models
print(tf.__version__)# 2.3.0

2.4.0-dev20200916


In [4]:
drive_path = "./drive/My Drive/camera_checker/step1-1"
train_dir = os.path.join(drive_path + "/train")
val_dir =  os.path.join(drive_path + "/val")
test_dir = os.path.join(drive_path + "/test")

saved_dir_name = drive_path + "/v2_3categories_epoch1000_val_loss_none"
save_model_path = saved_dir_name + "/model" 

In [5]:
print(os.listdir(train_dir))
print(os.listdir(val_dir))
print(os.listdir(test_dir))

['out', 'no_camera', 'have_camera']
['no_camera', 'out', 'have_camera']
['no_camera', 'have_camera', 'out']


ディレクトリ作成用の関数の作成

In [6]:
def make_dir(path):
    if not os.path.exists(path):
            print('make ', path)
            os.mkdir(path)

# train事前準備

In [7]:
BATCH_SIZE = 32
IMG_SIZE =224

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=(IMG_SIZE,IMG_SIZE))

validation_dataset = image_dataset_from_directory(val_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=(IMG_SIZE,IMG_SIZE))

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            image_size=(IMG_SIZE,IMG_SIZE))

Found 2100 files belonging to 3 classes.
Found 250 files belonging to 3 classes.
Found 250 files belonging to 3 classes.


# ラベルの表示

In [8]:
class_names = train_dataset.class_names
print(class_names)

['have_camera', 'no_camera', 'out']


In [9]:
print("train: ", train_dataset.class_names)
print("val: ",validation_dataset.class_names)
print("test: ",test_dataset.class_names)

train:  ['have_camera', 'no_camera', 'out']
val:  ['have_camera', 'no_camera', 'out']
test:  ['have_camera', 'no_camera', 'out']


In [10]:
n_categories = len(class_names)
print(n_categories)

3


In [11]:
# 処理が時間がかかるため省略

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

# パフォーマンスのためのデータセット構成
- トレーニングステップの前処理とモデル実行をオーバーラップ
- 実行中はステップの一つ先を読み取る

- データが生成される時間とデータが消費される時間を分離するために使用

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
print("train: ", train_dataset)
print("val: ", validation_dataset)
print("test: ", test_dataset)

train:  <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
val:  <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
test:  <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>


# データ拡張の指定
- 上記のtf.keras.layers.experimental.preprocessingでエラーが出た場合は
tf-nightlyでconflictが発生しているので以下のコマンドを実行
- !pip uninstall tf-nightly
- !pip install tensorflow --upgrade --force-reinstall

In [13]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [14]:
print(tf.__version__)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
  tf.keras.layers.experimental.preprocessing.RandomZoom(.2, .2),
])

2.4.0-dev20200916


In [15]:
# 時間がかかるので削除

# print(train_dataset.take(1))
# for image, _ in train_dataset.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

# Model定義

入力層の定義

In [16]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE) + (3,)
print(IMG_SHAPE)

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)


(224, 224, 3)


In [17]:
# base_model = tf.keras.applications.MobileNetV2(input_tensor=x,
#                                                 input_shape=IMG_SHAPE,
#                                                 include_top=False,
#                                                 weights='imagenet')

base_model = tf.keras.applications.MobileNetV2(input_tensor=x,
                                                input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights=None)

In [18]:
# block_15_add (Add)   まで凍結
# for layer in base_model.layers[:146]:
#     layer.trainable=False

In [19]:
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')

flatten_layer = tf.keras.layers.Flatten()
dense_layer1 = tf.keras.layers.Dense(256, activation = 'relu')
dropout_layer = tf.keras.layers.Dropout(0.5)

if n_categories == 2:
    dense_layer2  =tf.keras.layers.Dense(1, activation='sigmoid')
    print("sigmoid")
else:
    dense_layer2  =tf.keras.layers.Dense(n_categories, activation='softmax')
    print("softmax")

softmax


In [20]:
# x = base_model.output
# x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# outputs = prediction_layer(x)
# model = tf.keras.Model(inputs, outputs)

x = base_model.output
x = flatten_layer(x)
x = dense_layer1(x)
x = dropout_layer(x)
outputs = dense_layer2(x)
model = tf.keras.Model(inputs, outputs)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.math.truediv (TFOpLambda)    (None, 224, 224, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
tf.math.subtract (TFOpLambda)   (None, 224, 224, 3)  0           tf.math.truediv[0][0]            
______________________________________________________________________________________________

## モデルの学習

In [22]:
base_learning_rate = 1e-4
if n_categories == 2:
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                # optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                #   loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    print('binary_crossentropy')
    
else:
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    print('sparse_categorical_crossentropy')

sparse_categorical_crossentropy


# 特徴抽出

モデル保存用のディレクトリの作成

In [23]:
make_dir(saved_dir_name)
make_dir(save_model_path)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.math.truediv (TFOpLambda)    (None, 224, 224, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
tf.math.subtract (TFOpLambda)   (None, 224, 224, 3)  0           tf.math.truediv[0][0]            
______________________________________________________________________________________________

1.2Kのトレーニング可能なパラメータがある。
これらはtf.Variableオブジェクトである重みとバイアスに分けられる

In [25]:
len(model.trainable_variables)

160

In [26]:
initial_epochs = 1000

loss0, accuracy0 = model.evaluate(validation_dataset)

8/8 [==============================] - 134s 1s/step - loss: 1.0986 - accuracy: 0.4262


In [27]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 1.10
initial accuracy: 0.40


In [28]:
make_dir(save_model_path)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

model_earlystopping = EarlyStopping(monitor='val_loss', patience=10, mode='auto')
model_checkpoint = ModelCheckpoint(filepath=os.path.join(save_model_path , 'model_{epoch:03d}_val_loss_{val_loss:.4f}.h5'),
                            monitor='val_loss',
                            save_best_only=True,
                            save_weights_only=False,
                            mode='auto',
                            verbose=1,
                            period=1)
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    callbacks=[model_earlystopping,model_checkpoint])


Epoch 1/1000
 8/66 [==>...........................] - ETA: 15:09 - loss: 25.7188 - accuracy: 0.3711

学習したモデルの保存

In [ ]:
model.save(save_model_path+'/last_model.h5')

学習結果の出力

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

モデルの選択

In [ ]:
from tensorflow.keras.models import load_model
save_model_lists = os.listdir(save_model_path)
print(os.listdir(save_model_path)) 

# モデルを選択して実行したい場合はこのコメントアウトを消す
used_model = input('used model ...')
# used_model = save_model_lists[-2]
print(used_model)

model = load_model(os.path.join(save_model_path, used_model))

**テスト実行結果**

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

# ヒートマップの作成
[Visualizing Activation Heatmaps using TensorFlow \| by Areeb Gani \| Analytics Vidhya \| Medium](https://medium.com/analytics-vidhya/visualizing-activation-heatmaps-using-tensorflow-5bdba018f759)

In [ ]:
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K

def grad_cam(input_model,x,layer_name):
    X = np.expand_dims(x,axis=0)
    preprocess_input = X.astype('float32') / 255.0
    grad_model = models.Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(preprocess_input)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    # 勾配を計算
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')

    guided_grads = gate_f * gate_r * grads

    weights = tf.reduce_mean(guided_grads, axis=(0, 1))

    cam = np.ones(output.shape[0: 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam.numpy(), (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    # モノクロ画像に疑似的に色をつける
    jet_cam = cv2.applyColorMap(np.uint8(255.0*heatmap), cv2.COLORMAP_JET)
    # RGBに変換
    rgb_cam = cv2.cvtColor(jet_cam, cv2.COLOR_BGR2RGB)
    # もとの画像に合成
    output_image = (np.float32(rgb_cam) + x / 2)  

    return output_image


In [ ]:
# # kerasとtensorflowでGrad-CAMを実装してみた - Qiita https://qiita.com/Kensuke-N/items/cea3920f19bebc9e0d19
# from tensorflow.keras import preprocessing
# from tensorflow.keras import backend as K
# from tensorflow.keras import models

# # import tensorflow as tf
# # import numpy as np

# def grad_cam2(input_model,x,layer_name):



#     # 画像の前処理
#     # 読み込む画像が1枚なため、次元を増やしておかないとmode.predictが出来ない
#     X = np.expand_dims(x,axis=0)
#     preprocess_input = X.astype('float32') / 255.0

#     print(preprocess_input)

#     grad_model = models.Model([model.inputs], [input_model.get_layer(layer_name).output, input_model.output])

#     with tf.GradientTape() as tape:
#         conv_outputs, predictions = grad_model(preprocess_input)
#         print("conv_outputs", conv_outputs)
#         print("predictions", predictions)
#         class_idx = np.argmax(predictions[0])
        
#         print("class_idx",class_idx)
#         loss = predictions[:,class_idx]
#         #print(loss)
#     output = conv_outputs[0]
#     grads = tape.gradient(loss, conv_outputs)[0]

#     gate_f = tf.cast(output > 0, 'float32')
#     gate_r = tf.cast(grads > 0, 'float32')
#     guided_grads = gate_f * gate_r * grads
#     print(guided_grads.shape)
#     weights = tf.reduce_mean(guided_grads, axis=(0, 1))
#     cam = np.dot(output, weights)

#     # 画像を元画像と同じ大きさにスケーリング
#     cam = cv2.resize(cam, (224,224), cv2.INTER_LINEAR)
#     # ReLUの代わり
#     cam  = np.maximum(cam, 0)
#     # ヒートマップを計算
#     heatmap = cam / cam.max()

#     # モノクロ画像に疑似的に色をつける
#     jet_cam = cv2.applyColorMap(np.uint8(255.0*heatmap), cv2.COLORMAP_JET)
#     # RGBに変換
#     rgb_cam = cv2.cvtColor(jet_cam, cv2.COLOR_BGR2RGB)
#     # もとの画像に合成
#     output_image = (np.float32(rgb_cam) + x / 2)  

#     return output_image

In [ ]:
target_layer = 'Conv_1'
heatmap_path = os.path.join(saved_dir_name, 'heatmap_result')
make_dir(heatmap_path)

image_make = 1


for label_name in class_names:
    i = 0

    one_label_path = os.path.join(test_dir,label_name)
    one_label_heatmap_path = os.path.join(heatmap_path,label_name)

    make_dir(one_label_heatmap_path)
    one_images_lists = os.listdir(one_label_path)
    for one_image in one_images_lists:
        print("i= ", i)
        if i < image_make:
            heatmap_name = str(one_label_heatmap_path) +'/heatmap_' +str(one_image)
            one_image_path = os.path.join(one_label_path,one_image)
            x = img_to_array(load_img(one_image_path,target_size=(IMG_SIZE,IMG_SIZE) ))
            cam = grad_cam(model,x,target_layer)
            cv2.imwrite(heatmap_name, cam)
            i += 1
        else:
            break
        

#TODO
- classification_report